[Artigo: CDRL](https://drive.google.com/drive/u/0/folders/1NR9Ty7_5lV8Wpn0mNB2PUx1bozJZ-hVm)

In [2]:
import numpy as np
from scipy.special import softmax
# import pandas as pd

# Definição do Esimador:

$\hat{T}_m(s,a,k) = \hat{T}_m(s,a,k) + \Delta \hat{T}_m(k) , \; \forall k \in S$ 

$
    \Delta \hat{T}_m(k) = 
        \begin{cases}
            \frac{1- \hat{T}_m(s,a,k) }{N_m(s,a)+1} , & \text{if } k = s'\\
            \\
            \frac{0- \hat{T}_m(s,a,k) }{N_m(s,a)+1}, & \text{if } k \neq s'
        \end{cases} 
$  



$\hat{R}_m(s,a,s') = \hat{R}_m(s,a,s') + \Delta \hat{R}_m $ 

$\Delta \hat{R}_m = \frac{r- \hat{R}_m(s,a) }{N_m(s,a)+1}$  


### Qualidade Instantanea por estimador

$e^T_{m} = 1-2(Z_T \sum_{k \in S}\Delta\hat{T}_m(k)^2)$ 

$e^R_{m} = 1-2(Z_R \Delta\hat{R}_m^2)$ 


$Z_T= \frac{1}{2}(N(s,a)+1)^2$ \
$Z_R= (R_{\max}-R_{\min})^{-1} = (0 - (-1))^{-1} = 1$  


In [3]:
class Estimator:
    def __init__(self, M=1e6, *inputs):
        self.M = M
        self.n = np.zeros(inputs)     
        self.v = np.zeros(inputs) 
        # self.e = 0

    def e(self, value=1, *inputs):
        return 1-2 * (self.z(*inputs) * self.delta(value, *inputs)**2)

    def z(self, *inputs):
        return 1

    def delta(self, value, *inputs):
        return (value-self.v[inputs]) / (np.sum(self.n[inputs[:-1]])+1)

    def train(self, value=1, *inputs):
        self.n[inputs] = min(self.n[inputs]+1, self.M)
        delta = self.delta(value, *inputs)
        self.v[inputs] += delta
        
    def forward(self, *inputs):
        return self.v[inputs]
    
    
    
est = Estimator((5,3))

In [4]:
class Estimator_T(Estimator):    
    def e(self, value=1, *inputs):
        return 1-2*  self.z(*inputs)  * np.sum(self.delta(value, *inputs)**2)
    
    def z(self, *inputs):
        s,a,_ = inputs
        return (1/2)*(np.sum(self.n[s,a])+1)**2    

    def delta(self, value=1, *inputs):
        S = self.n.shape[0]
        s,a,s_ = inputs
        d = lambda k: (
                (value-self.v[s,a,k])/(np.sum(self.n[s,a])+1) 
                    if k==s_ else 
                (0-self.v[s,a,k])/(np.sum(self.n[s,a])+1)
            )
        return np.array([d(k) for k in range(S)])

    
    def train(self, value=1, *inputs):
        S = self.n.shape[0]
        s,a,s_ = inputs

        self.n[inputs] = min(self.n[inputs]+1, self.M)
        delta = self.delta(value, *inputs)
        for k in range(S):
            self.v[s,a,k] += delta[k]

# Modelo
#### Qualidade Instantanea modelo:  


$E_m = E_m + \rho(e_m - E_m)$

$\rho = 1$

$e_m = c_m(s,a) (\Omega e^R_m + (1-\Omega)e^T_m)$

$c_m(s,a) = \frac{N_m(s,a)}{M}$  
$\Omega = 0$



In [5]:
class Model:
    def __init__(self, S,A, Omega=.5, M=1e2, rho=.9):
        self.S = S
        self.A = A
        self.Omega = Omega
        self.M = M
        self.rho = rho
        self.N = 0

        self._E = 0 

        self.t = Estimator_T(self.M, len(self.S), len(self.A) ,len(self.S))
        self.r = Estimator(self.M, len(self.S), len(self.A) ,len(self.S))

    def e(self, s,a,s_,r):
        nm = min(np.sum(self.t.n[s,a])+1, self.t.M)
        cm = nm/self.t.M
        return cm*(self.Omega * self.r.e(r, s,a,s_) + (1-self.Omega) * self.t.e(1, s,a,s_))
    
    def E(self, s,a,s_,r):
        self._E += self.rho*(self.e(s,a,s_,r) - self._E)
        return self._E

    def learn(self, s,a,s_,r):
        self.N += 1 
        self.t.train(1, s,a,s_)
        self.r.train(r, s,a,s_)
    
    def simulate(self, s, a): 
        s_ = np.random.choice(len(self.S), p=softmax(self.t.forward(s,a)))
        r = self.r.forward(s,a,s_)
        return s_,r
        
    def T(self, s,a,s_): # predict_T
        return self.t.forward(s,a,s_)
    def R(self, s,a,s_):
        return self.r.forward(s,a,s_)

model = Model(range(10), range(4))

In [6]:
class RLCD:
    def __init__(self, S,A, Emin=-0.01, Omega=0, M=1e2):
        self.S = S
        self.A = A
        self.Models = []
        self.current_model = None
        self.Emin = Emin
        self.Omega = Omega
        self.M = M
        self.new_model()

    def new_model(self, M=1e2):
        self.current_model = Model(self.S, self.A, self.Omega, self.M)
        self.Models.append(self.current_model)

    def learn(self, s,a,s_,r, log=False):
        E = [m.E(s,a,s_,r) for m in self.Models]
        self.current_model = self.Models[ np.argmax(E) ]
        new_model = self.current_model._E < self.Emin
        
        if log:
            print('E: ', E, new_model)
        if new_model:
            self.new_model()

        self.current_model.learn(s,a,s_,r)
    
    def simulate(self, s, a):
        sims = [m.simulate(s,a) for m in self.Models]
        E = [m.E(s,a,s_,r) for m, (s_,r) in zip(self.Models, sims)]
        return sims[ np.argmax(E) ]


model = RLCD(range(10), range(4))

# Simulação

In [7]:
p1, p2 = .8, .5
n = 1000

d1 = np.random.choice(2, n, p=[1-p1, p1])
d2 = np.random.choice(2, n, p=[1-p2, p2])

data = np.concatenate([d1,d2])
print(data.sum()/data.size)
print(data)

0.642
[1 1 1 ... 0 1 0]


In [8]:
(p1 + p2)/2

0.65

In [12]:
est1 = Estimator_T(1e6, 2, 1, 2)
est2 = Estimator_T(1e6, 2, 1, 2)
s = None
for i, s_ in enumerate(data):
    if s is None:
        pass
    else:
        if i < n:
            est1.train(1, s,0,s_)
        else:
            est2.train(1, s,0,s_)
        # print(s, s_, ':',round(est1.e(1,s,0,s_), 3), round(est2.e(1,s,0,s_), 3))

    s = s_

print("Models:")
print(est1.forward(0, 0, 1))
print(est2.forward(0, 0, 1))

Models:
0.7743589743589744
0.4865900383141763


In [13]:
model = RLCD([0,1], [0])

s = None
for s_ in data:
    if s is None:
        pass
    else:
        model.learn(s,0,s_,0, log=True)
    s = s_

model.simulate(0,0)[0]

E:  [0.0] False
E:  [0.0135] False
E:  [-0.010650000000000001] True
E:  [-0.001065, 0.0] False
E:  [0.023893499999999998, -0.0045000000000000005] False
E:  [-0.01786065, 0.013049999999999999] False
E:  [-0.0017860649999999999, 0.014804999999999999] False
E:  [0.03357139349999999, -0.010519500000000001] False
E:  [0.04655713935, -0.01305195] False
E:  [0.05715571393499999, -0.013305194999999999] False
E:  [0.06742985710778572, -0.013330519499999999] False
E:  [0.07761798571077858, -0.013333051949999999] False
E:  [0.08776179857107785, -0.013333305194999999] False
E:  [0.0978761798571078, -0.0133333305195] False
E:  [0.10796943616752896, -0.01333333305195] False
E:  [0.1180469436167529, -0.013333333305194999] False
E:  [0.1281123866693676, -0.0133333333305195] False
E:  [0.13816838152407962, -0.01333333333305195] False
E:  [0.14821683815240794, -0.013333333333305194] False
E:  [0.15825918381524082, -0.013333333333330519] False
E:  [0.1682965066168182, -0.013333333333333051] False
E:  [0.

1

In [16]:
[m.N for m in model.Models]

[878, 811, 310]

In [18]:
n = 1000
sum([model.simulate(0,0)[0] for _ in range(n)])/n

0.511

In [21]:
n = 1000
[
    sum([m.simulate(0,0)[0] for _ in range(n)])/n
    for m in model.Models
]

[0.505, 0.742, 0.256]